In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, StandardScaler

so_survey_csv = 'https://assets.datacamp.com/production/repositories/3752/datasets/19699a2441073ad6459bf5e3e17690e2cae86cf1/Combined_DS_v10.csv'
so_survey_df = pd.read_csv(so_survey_csv)

speech_df = pd.read_csv("../../data/inaugural_speeches.csv")

## Cleaning up your text
### Unstructured text data cannot be directly used in most analyses. Multiple steps need to be taken to go from a long free form string to a set of numeric columns in the right format that can be ingested by a machine learning model. The first step of this process is to standardize the data and eliminate any characters that could cause problems later on in your analytic pipeline.

### In this chapter you will be working with a new dataset containing the inaugural speeches of the presidents of the United States loaded as speech_df, with the speeches stored in the text column.

### Instructions 1/2
-    Print the first 5 rows of the text column to see the free text fields.

In [3]:
# Print the first 5 rows of the text column
print(speech_df["text"].head())

0    Fellow-Citizens of the Senate and of the House...
1    Fellow Citizens:  I AM again called upon by th...
2    WHEN it was first perceived, in early times, t...
3    Friends and Fellow-Citizens:  CALLED upon to u...
4    PROCEEDING, fellow-citizens, to that qualifica...
Name: text, dtype: object


### Instructions 2/2
-    Replace all non letter characters in the text column with a whitespace.
-    Make all characters in the newly created text_clean column lower case.

In [4]:
# Replace all non letter characters with a whitespace
speech_df['text_clean'] = speech_df['text'].str.replace('[^a-zA-Z]', ' ')

# Change to lower case
speech_df['text_clean'] = speech_df['text_clean'].str.lower()

# Print the first 5 rows of the text_clean column
print(speech_df['text_clean'].head())

0    fellow-citizens of the senate and of the house...
1    fellow citizens:  i am again called upon by th...
2    when it was first perceived, in early times, t...
3    friends and fellow-citizens:  called upon to u...
4    proceeding, fellow-citizens, to that qualifica...
Name: text_clean, dtype: object


## High level text features
### Once the text has been cleaned and standardized you can begin creating features from the data. The most fundamental information you can calculate about free form text is its size, such as its length and number of words. In this exercise (and the rest of this chapter), you will focus on the cleaned/transformed text column (text_clean) you created in the last exercise.

### Instructions
-    Record the character length of each speech in the char_count column.
-    Record the word count of each speech in the word_count column.
-    Record the average word length of each speech in the avg_word_length column.

In [5]:
# Find the length of each text
speech_df['char_cnt'] = speech_df['text_clean'].str.len()

# Count the number of words in each text
speech_df['word_cnt'] = speech_df['text_clean'].str.split().str.len()

# Find the average length of word
speech_df['avg_word_length'] = speech_df['char_cnt'] / speech_df['word_cnt']

# Print the first 5 rows of these columns
print(speech_df[['text_clean', 'char_cnt', 'word_cnt', 'avg_word_length']])

                                           text_clean  char_cnt  word_cnt  \
0   fellow-citizens of the senate and of the house...      8616      1427   
1   fellow citizens:  i am again called upon by th...       787       135   
2   when it was first perceived, in early times, t...     13871      2317   
3   friends and fellow-citizens:  called upon to u...     10144      1717   
4   proceeding, fellow-citizens, to that qualifica...     12902      2157   
5   unwilling to depart from examples of the most ...      7003      1173   
6   about to add the solemnity of an oath to the o...      7148      1210   
7   i should be destitute of feeling if i was not ...     19894      3367   
8   fellow-citizens:  i shall not attempt to descr...     26322      4462   
9   in compliance with an usage coeval with the ex...     17753      2907   
10  fellow-citizens:  about to undertake the arduo...      6818      1124   
11  fellow-citizens:  the will of the american peo...      7061      1171   

## Counting words (I)
### Once high level information has been recorded you can begin creating features based on the actual content of each text. One way to do this is to approach it in a similar way to how you worked with categorical variables in the earlier lessons.

### For each unique word in the dataset a column is created.
### For each entry, the number of times this word occurs is counted and the count value is entered into the respective column.
### These "count" columns can then be used to train machine learning models.

### Instructions
-    Import CountVectorizer from sklearn.feature_extraction.text.
-    Instantiate CountVectorizer and assign it to cv.
-    Fit the vectorizer to the text_clean column.
-    Print the feature names generated by the vectorizer.

In [9]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate CountVectorizer
cv = CountVectorizer()

# Fit the vectorizer
cv.fit(speech_df['text_clean'])

# Print feature names
#print(cv.get_feature_names()) ## Function doesn't seem to be there
print(cv.get_feature_names_out())

['0085' '0092' '0093' ... 'zealous' 'zealously' 'zone']


## Counting words (II)
### Once the vectorizer has been fit to the data, it can be used to transform the text to an array representing the word counts. This array will have a row per block of text and a column for each of the features generated by the vectorizer that you observed in the last exercise.

### The vectorizer to you fit in the last exercise (cv) is available in your workspace.

### Instructions 1/2
-    Apply the vectorizer to the text_clean column.
-    Convert this transformed (sparse) array into a numpy array with counts.

In [10]:
# Apply the vectorizer
cv_transformed = cv.transform(speech_df['text_clean'])

# Print the full array
cv_array = cv_transformed.toarray()
print(cv_array)

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 3 12  1 ...  0  0  0]
 [ 0 12  1 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]]


### Instructions 2/2
-    Print the dimensions of this numpy array.

In [11]:
# Apply the vectorizer
cv_transformed = cv.transform(speech_df['text_clean'])

# Print the full array
cv_array = cv_transformed.toarray()

# Print the shape of cv_array
print(cv_array.shape)

(58, 9048)


## Limiting your features
### As you have seen, using the CountVectorizer with its default settings creates a feature for every single word in your corpus. This can create far too many features, often including ones that will provide very little analytical value.

### For this purpose CountVectorizer has parameters that you can set to reduce the number of features:

### -    min_df : Use only words that occur in more than this percentage of documents. This can be used to remove outlier words that will not generalize across texts.
### -    max_df : Use only words that occur in less than this percentage of documents. This is useful to eliminate very common words that occur in every corpus without adding value such as "and" or "the".
### Instructions
-    Limit the number of features in the CountVectorizer by setting the minimum number of documents a word can appear to 20% and the maximum to 80%.
-    Fit and apply the vectorizer on text_clean column in one step.
-    Convert this transformed (sparse) array into a numpy array with counts.
-    Print the dimensions of the new reduced array.

In [12]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Specify arguements to limit the number of features generated
cv = CountVectorizer(min_df=0.2, max_df=0.8)

# Fit, transform, and convert into array
cv_transformed = cv.fit_transform(speech_df['text_clean'])
cv_array = cv_transformed.toarray()

# Print the array shape
print(cv_array.shape)

(58, 818)


## Text to DataFrame
### Now that you have generated these count based features in an array you will need to reformat them so that they can be combined with the rest of the dataset. This can be achieved by converting the array into a pandas DataFrame, with the feature names you found earlier as the column names, and then concatenate it with the original DataFrame.

### The numpy array (cv_array) and the vectorizer (cv) you fit in the last exercise are available in your workspace.

### Instructions
-    Create a DataFrame cv_df containing the cv_array as the values and the feature names as the column names.
-    Add the prefix Counts_ to the column names for ease of identification.
-    Concatenate this DataFrame (cv_df) to the original DataFrame (speech_df) column wise.

In [14]:
# Create a DataFrame with these features
#cv_df = pd.DataFrame(cv_array, 
#                     columns=cv.get_feature_names()).add_prefix('Counts_')
cv_df = pd.DataFrame(cv_array, 
                     columns=cv.get_feature_names_out()).add_prefix('Counts_')

# Add the new columns to the original DataFrame
speech_df_new = pd.concat([speech_df, cv_df], axis=1, sort=False)
print(speech_df_new.head())

                Name         Inaugural Address                      Date  \
0  George Washington   First Inaugural Address  Thursday, April 30, 1789   
1  George Washington  Second Inaugural Address     Monday, March 4, 1793   
2         John Adams         Inaugural Address   Saturday, March 4, 1797   
3   Thomas Jefferson   First Inaugural Address  Wednesday, March 4, 1801   
4   Thomas Jefferson  Second Inaugural Address     Monday, March 4, 1805   

                                                text  \
0  Fellow-Citizens of the Senate and of the House...   
1  Fellow Citizens:  I AM again called upon by th...   
2  WHEN it was first perceived, in early times, t...   
3  Friends and Fellow-Citizens:  CALLED upon to u...   
4  PROCEEDING, fellow-citizens, to that qualifica...   

                                          text_clean  char_cnt  word_cnt  \
0  fellow-citizens of the senate and of the house...      8616      1427   
1  fellow citizens:  i am again called upon by th...  

## Tf-idf
### While counts of occurrences of words can be useful to build models, words that occur many times may skew the results undesirably. To limit these common words from overpowering your model a form of normalization can be used. In this lesson you will be using Term frequency-inverse document frequency (Tf-idf) as was discussed in the video. Tf-idf has the effect of reducing the value of common words, while increasing the weight of words that do not occur in many documents.

### Instructions
-    Import TfidfVectorizer from sklearn.feature_extraction.text.
-    Instantiate TfidfVectorizer while limiting the number of features to 100 and removing English stop words.
-    Fit and apply the vectorizer on text_clean column in one step.
-    Create a DataFrame tv_df containing the weights of the words and the feature names as the column names.

In [16]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer
tv = TfidfVectorizer(max_features=100, stop_words='english')

# Fit the vectroizer and transform the data
tv_transformed = tv.fit_transform(speech_df['text_clean'])

# Create a DataFrame with these features
#tv_df = pd.DataFrame(tv_transformed.toarray(), 
#                     columns=tv.get_feature_names()).add_prefix('TFIDF_')
tv_df = pd.DataFrame(tv_transformed.toarray(), 
                     columns=tv.get_feature_names_out()).add_prefix('TFIDF_')
print(tv_df.head())

   TFIDF_0092  TFIDF_0097  TFIDF_action  TFIDF_administration  TFIDF_america  \
0         0.0    0.047468      0.000000              0.133265       0.000000   
1         0.0    0.000000      0.000000              0.261016       0.266097   
2         0.0    0.021955      0.000000              0.092456       0.157092   
3         0.0    0.131111      0.000000              0.092023       0.000000   
4         0.0    0.028455      0.041523              0.039943       0.000000   

   TFIDF_american  TFIDF_americans  TFIDF_believe  TFIDF_best  TFIDF_better  \
0        0.105269              0.0       0.000000    0.000000      0.000000   
1        0.000000              0.0       0.000000    0.000000      0.000000   
2        0.073033              0.0       0.000000    0.026118      0.060473   
3        0.000000              0.0       0.090286    0.116980      0.045143   
4        0.031552              0.0       0.000000    0.067701      0.039189   

   ...  TFIDF_things  TFIDF_time  TFIDF_toda

## Inspecting Tf-idf values
### After creating Tf-idf features you will often want to understand what are the most highest scored words for each corpus. This can be achieved by isolating the row you want to examine and then sorting the the scores from high to low.

### The DataFrame from the last exercise (tv_df) is available in your workspace.

### Instructions
-    Assign the first row of tv_df to sample_row.
-    sample_row is now a series of weights assigned to words. Sort these values to print the top 5 highest-rated words.

In [17]:
# Isolate the row to be examined
sample_row = tv_df.iloc[0]

# Print the top 5 words of the sorted output
print(sample_row.sort_values(ascending=False).head())

TFIDF_government    0.367016
TFIDF_public        0.332862
TFIDF_present       0.314827
TFIDF_duty          0.238368
TFIDF_country       0.229385
Name: 0, dtype: float64


## Transforming unseen data
### When creating vectors from text, any transformations that you perform before training a machine learning model, you also need to apply on the new unseen (test) data. To achieve this follow the same approach from the last chapter: fit the vectorizer only on the training data, and apply it to the test data.

### For this exercise the speech_df DataFrame has been split in two:

### -    train_speech_df: The training set consisting of the first 45 speeches.
### -    test_speech_df: The test set consisting of the remaining speeches.
### Instructions
-    Instantiate TfidfVectorizer.
-    Fit the vectorizer and apply it to the text_clean column.
-    Apply the same vectorizer on the text_clean column of the test data.
-    Create a DataFrame of these new features from the test set.

In [23]:
train_speech_df = speech_df_new[:45]
test_speech_df = speech_df_new[45:]

In [24]:
# Instantiate TfidfVectorizer
tv = TfidfVectorizer(max_features=100, stop_words='english')

# Fit the vectroizer and transform the data
tv_transformed = tv.fit_transform(train_speech_df['text_clean'])

# Transform test data
test_tv_transformed = tv.transform(test_speech_df['text_clean'])

# Create new features for the test set
#test_tv_df = pd.DataFrame(test_tv_transformed.toarray(), 
#                          columns=tv.get_feature_names()).add_prefix('TFIDF_')
test_tv_df = pd.DataFrame(test_tv_transformed.toarray(), 
                          columns=tv.get_feature_names_out()).add_prefix('TFIDF_')
print(test_tv_df.head())

   TFIDF_0097  TFIDF_action  TFIDF_administration  TFIDF_america  \
0    0.451720      0.000000              0.026354       0.208724   
1    0.264367      0.000000              0.000000       0.528140   
2    0.426195      0.000000              0.000000       0.114876   
3    0.289710      0.035503              0.064536       0.255561   
4    0.210826      0.000000              0.000000       0.216971   

   TFIDF_american  TFIDF_authority  TFIDF_best  TFIDF_business  \
0        0.073784         0.000000    0.000000        0.000000   
1        0.035562         0.000000    0.034765        0.000000   
2        0.121826         0.000000    0.119096        0.000000   
3        0.030114         0.038275    0.058878        0.047937   
4        0.153399         0.027853    0.085692        0.000000   

   TFIDF_citizens  TFIDF_commerce  ...  TFIDF_subject  TFIDF_support  \
0        0.020142             0.0  ...            0.0       0.000000   
1        0.014562             0.0  ...            

## Using longer n-grams
### So far you have created features based on individual words in each of the texts. This can be quite powerful when used in a machine learning model but you may be concerned that by looking at words individually a lot of the context is being ignored. To deal with this when creating models you can use n-grams which are sequence of n words grouped together. For example:

### -    bigrams: Sequences of two consecutive words
### -    trigrams: Sequences of two consecutive words
### These can be automatically created in your dataset by specifying the ngram_range argument as a tuple (n1, n2) where all n-grams in the n1 to n2 range are included.

### Instructions
-    Import CountVectorizer from sklearn.feature_extraction.text.
-    Instantiate CountVectorizer while considering only trigrams.
-    Fit the vectorizer and apply it to the text_clean column in one step.
-    Print the feature names generated by the vectorizer.

In [26]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate a trigram vectorizer
cv_trigram_vec = CountVectorizer(max_features=100, 
                                 stop_words='english', 
                                 ngram_range = (3,3))

# Fit and apply trigram vectorizer
cv_trigram = cv_trigram_vec.fit_transform(speech_df['text_clean'])

# Print the trigram features
#print(cv_trigram_vec.get_feature_names())
print(cv_trigram_vec.get_feature_names_out())

['0092 ideal freedom' 'ability preserve protect'
 'agriculture commerce manufactures' 'america 0092 ideal'
 'best ability preserve' 'best interests country' 'bless god bless'
 'bless united states' 'chief justice mr' 'children children children'
 'citizens united states' 'civil religious liberty'
 'commerce united states' 'concern thank god' 'confidence fellow citizens'
 'congress extraordinary session' 'constitution does expressly'
 'constitution united states' 'coordinate branches government'
 'day task people' 'defend constitution united' 'desire determined work'
 'distinguished guests fellow' 'does expressly say' 'equal exact justice'
 'era good feeling' 'executive branch government'
 'faithfully execute office' 'fellow citizens assembled'
 'fellow citizens called' 'fellow citizens large' 'fellow citizens world'
 'form perfect union' 'general welfare secure' 'god bless america'
 'god bless god' 'good greatest number' 'government united states'
 'granted federal government' 'great b

## Finding the most common words
### Its always advisable once you have created your features to inspect them to ensure that they are as you would expect. This will allow you to catch errors early, and perhaps influence what further feature engineering you will need to do.

### The vectorizer (cv) you fit in the last exercise and the sparse array consisting of word counts (cv_trigram) is available in your workspace.

### Instructions
-    Create a DataFrame of the features (word counts).
-    Add the counts of word occurrences and print the top 5 most occurring words.

In [28]:
# Create a DataFrame of the features
#cv_tri_df = pd.DataFrame(cv_trigram.toarray(), 
#                 columns=cv_trigram_vec.get_feature_names()).add_prefix('Counts_')
cv_tri_df = pd.DataFrame(cv_trigram.toarray(), 
                 columns=cv_trigram_vec.get_feature_names_out()).add_prefix('Counts_')

# Print the top 5 words in the sorted output
print(cv_tri_df.sum().sort_values(ascending=False).head())

Counts_constitution united states    20
Counts_people united states          13
Counts_mr chief justice              10
Counts_preserve protect defend       10
Counts_president united states        8
dtype: int64
